# 🔄 Migration Plan: From Notebook to Production App

## Current Status: Proof of Concept (Jupyter Notebook) ✅
This notebook serves as a functional POC but has limitations for production use:
- Monolithic structure makes maintenance difficult
- Hard to version control and collaborate on
- Not suitable for production deployment
- Limited testing capabilities

## ✅ COMPLETED: Production Application Created!

### 📁 New Application Location: `build_buy_app/`
A production-ready Dash application has been created in the `build_buy_app` directory with:

- **Modular Structure**: Separate files for different concerns
- **Configuration-Driven**: Easy to add new parameters
- **Testable**: Basic testing framework included
- **Production-Ready**: Can be deployed to servers
- **Maintainable**: Easy to modify and extend

### 🚀 To Run the Production App:
```bash
cd build_buy_app
pip install -r requirements.txt
python app.py
```

## Migration Benefits:
1. **Better Organization**: Code split into logical modules
2. **Version Control**: Git-friendly structure
3. **Collaboration**: Others can easily contribute
4. **Testing**: Verify functionality automatically
5. **Deployment**: Ready for production servers
6. **Maintenance**: Easy to add/remove features

## Next Steps for Your Team:
1. **Review** the new application structure
2. **Test** the functionality matches the notebook
3. **Deploy** to a shared server for team access
4. **Extend** with additional features as needed

---

**Note**: This notebook will remain as the working prototype and documentation, while the `build_buy_app` directory contains the production-ready version.

# Build vs. Buy Decision-Support Dashboard

This notebook is a proof-of-concept for a dashboard that helps stakeholders decide whether to build a technology solution in-house or buy it. The tool aims to reduce bias by using a data-driven approach, incorporating key business variables and decision analysis techniques.

### Imports

In [1]:
# Import necessary libraries for notebook operations
import os
import sys
import dash
import pandas as pd
import numpy as np
import plotly.express as px
import yfinance as yf
from dash import html, dcc, Input, Output, State
from dash.dependencies import Input, Output, State

/Users/jheyer/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. UI/Layout

In [2]:
# --- UI and App Layout ---
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = html.Div([
    html.H2("🛠️ Build vs. Buy Decision-Support Dashboard", style={
        "textAlign": "center",
        "marginBottom": "20px",
        "fontFamily": "Segoe UI, Arial, sans-serif",
        "color": "#2c3e50"
    }),
    dcc.Store(id="scenario_store", data=[]),  # Store for scenario data
    html.Div([
        html.Div([
            html.Div([
                html.H4("🔧 Build Cost Parameters", style={
                    "color": "#34495e",
                    "marginBottom": "10px"
                }),
                html.Label("Select Risk Factors to Include:"),
                dcc.Checklist(
                    id="risk_selector",
                    options=[
                        {"label": "Technical Risk", "value": "tech"},
                        {"label": "Vendor Risk", "value": "vendor"},
                        {"label": "Market Risk", "value": "market"}
                    ],
                    value=[],
                    inline=False,
                    style={
                        "marginBottom": "10px",
                        "display": "block",
                        "padding": "0",
                        "gap": "8px"
                    },
                    inputStyle={
                        "marginRight": "8px",
                        "marginBottom": "8px"
                    },
                    labelStyle={
                        "display": "block",
                        "marginBottom": "8px"
                    }
                ),
                html.Div(id="risk_inputs"),
                html.Label("Select Cost Parameters to Include:"),
                dcc.Checklist(
                    id="cost_selector",
                    options=[
                        {"label": "Annual Maintenance/OpEx", "value": "opex"},
                        {"label": "CapEx Investment", "value": "capex"},
                        {"label": "Monthly Amortization", "value": "amortization"}
                    ],
                    value=[],  # Start with no cost parameters checked
                    inline=False,
                    style={
                        "marginBottom": "10px",
                        "display": "block",
                        "padding": "0",
                        "gap": "8px"
                    },
                    inputStyle={
                        "marginRight": "8px",
                        "marginBottom": "8px"
                    },
                    labelStyle={
                        "display": "block",
                        "marginBottom": "8px"
                    }
                ),
                html.Div(id="cost_inputs"),
                html.Label("Build Timeline (months)", style={"marginTop": "10px"}),
                dcc.Input(id="build_timeline", type="number", value=12, step=1, style={"width": "100%"}),
                html.Label("Build Timeline Uncertainty (std dev, months)", style={"marginTop": "10px"}),
                dcc.Input(id="build_timeline_std", type="number", value=0, step=0.1, style={"width": "100%"}),
                html.Label("FTE Cost ($/yr)", style={"marginTop": "10px"}),
                dcc.Input(id="fte_cost", type="number", value=130000, step=1000, style={"width": "100%"}),
                html.Label("FTE Cost Uncertainty (std dev, $)", style={"marginTop": "10px"}),
                dcc.Input(id="fte_cost_std", type="number", value=15000, step=100, style={"width": "100%"}),
                html.Label("FTE Count", style={"marginTop": "10px"}),
                dcc.Input(id="fte_count", type="number", value=3, step=1, style={"width": "100%"}),
                html.Label("Capitalization Percent (%)", style={"marginTop": "10px"}),
                dcc.Input(id="cap_percent", type="number", value=75, step=1, style={"width": "100%"}),
                html.Label("Misc Costs ($)", style={"marginTop": "10px"}),
                dcc.Input(id="misc_costs", type="number", value=0, step=100, style={"width": "100%"}),
            ], style={
                "backgroundColor": "#f8f9fa",
                "borderRadius": "12px",
                "boxShadow": "0 2px 8px rgba(44,62,80,0.07)",
                "padding": "24px",
                "marginBottom": "20px",
                "border": "1px solid #e1e4e8"
            }),
            # ...existing code for Buy Cost Parameters, buttons, etc...
            html.Div([
                html.H4("💳 Buy Cost Parameters", style={
                    "color": "#34495e",
                    "marginBottom": "10px",
                    "marginTop": "20px"
                }),
                html.Label("Select Buy Cost Components:"),
                dcc.Checklist(
                    id="buy_selector",
                    options=[
                        {"label": "One-Time Purchase (Flat Fee)", "value": "one_time"},
                        {"label": "Annual Subscription", "value": "subscription"}
                    ],
                    value=[],
                    inline=False,
                    style={
                        "marginBottom": "10px",
                        "display": "block",
                        "padding": "0",
                        "gap": "8px"
                    },
                    inputStyle={
                        "marginRight": "8px",
                        "marginBottom": "8px"
                    },
                    labelStyle={
                        "display": "block",
                        "marginBottom": "8px"
                    }
                ),
                html.Div(id="buy_inputs"),
                html.Label("Useful Life (years)", style={"marginTop": "10px"}),
                dcc.Input(id="useful_life", type="number", value=5, step=1, style={"width": "100%"}),
            ], style={
                "backgroundColor": "#f8f9fa",
                "borderRadius": "12px",
                "boxShadow": "0 2px 8px rgba(44,62,80,0.07)",
                "padding": "24px",
                "marginBottom": "20px",
                "border": "1px solid #e1e4e8"
            }),
            # ...existing code for Probability of Success, WACC, Scenario Name, and buttons...
            html.Br(),
            html.Label("Probability of Success (%)", style={"marginTop": "10px"}),
            dcc.Input(id="prob_success", type="number", value=90, step=1, style={"width": "100%"}),
            html.Label("WACC (%)", style={"marginTop": "10px"}),
            dcc.Input(id="wacc", type="number", value=8, step=0.1, style={"width": "100%"}),
            html.Label("Scenario Name", style={"marginTop": "10px"}),
            dcc.Input(id="scenario_name", type="text", value="", placeholder="Enter scenario name", style={"width": "100%"}),
            html.Br(),
            html.Button("💡 Calculate", id="calc_btn", n_clicks=0, style={
                "marginTop": "20px",
                "backgroundColor": "#2980b9",
                "color": "white",
                "border": "none",
                "padding": "10px 24px",
                "fontSize": "16px",
                "borderRadius": "6px",
                "cursor": "pointer"
            }),
            html.Button("⬇️ Download Results (CSV)", id="download_btn", n_clicks=0, style={
                "marginTop": "10px",
                "backgroundColor": "#27ae60",
                "color": "white",
                "border": "none",
                "padding": "8px 20px",
                "fontSize": "15px",
                "borderRadius": "6px",
                "cursor": "pointer"
            }),
            html.Button("💾 Save Scenario", id="save_scenario_btn", n_clicks=0, style={
                "marginTop": "10px",
                "backgroundColor": "#f39c12",
                "color": "white",
                "border": "none",
                "padding": "8px 20px",
                "fontSize": "15px",
                "borderRadius": "6px",
                "cursor": "pointer"
            }),
            dcc.Download(id="download_csv")
        ], style={
            "width": "40%",
            "display": "inline-block",
            "verticalAlign": "top",
            "padding": "20px"
        }),
        # ...existing code for results, graph, and scenario table...
        html.Div([
            html.H4("📊 Results", style={
                "color": "#34495e",
                "marginBottom": "10px"
            }),
            html.Div(id="results", style={
                "backgroundColor": "#eaf6fb",
                "borderRadius": "12px",
                "boxShadow": "0 2px 8px rgba(44,62,80,0.07)",
                "padding": "24px",
                "border": "1px solid #b2bec3",
                "marginBottom": "20px",
                "color": "#000 !important",
                "fontWeight": "500"
            }),
            dcc.Graph(id="cost_dist", style={
                "backgroundColor": "#fff",
                "borderRadius": "12px",
                "boxShadow": "0 2px 8px rgba(44,62,80,0.07)",
                "padding": "12px"
            }),
            html.Div(id="scenario_table_container", style={"marginTop": "24px"}),
        ], style={
            "width": "55%",
            "display": "inline-block",
            "padding": "20px"
        })
    ])
] , style={
    "fontFamily": "Segoe UI, Arial, sans-serif",
    "backgroundColor": "#f4f6f8",
    "padding": "24px"
})

## 2. Dynamic Dependencies

In [3]:
from dash.dependencies import Input, Output

@app.callback(
    Output('risk_inputs', 'children'),
    [Input('risk_selector', 'value')]
 )
def display_risk_inputs(selected):
    # Technical Risk input always exists but hidden if not selected
    tech_div = html.Div([
        html.Label('Technical Risk (%)'),
        dcc.Input(id='tech_risk', type='number', value=0, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'tech' in selected else 'none'})
    # Vendor Risk input
    vendor_div = html.Div([
        html.Label('Vendor Risk (%)'),
        dcc.Input(id='vendor_risk', type='number', value=0, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'vendor' in selected else 'none'})
    # Market Risk input
    market_div = html.Div([
        html.Label('Market Risk (%)'),
        dcc.Input(id='market_risk', type='number', value=0, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'market' in selected else 'none'})
    return [tech_div, vendor_div, market_div]

@app.callback(
    Output('cost_inputs', 'children'),
    [Input('cost_selector', 'value')]
 )
def display_cost_inputs(selected):
    # Opex input always exists but hidden if not selected
    opex_div = html.Div([
        html.Label('Annual Maintenance/OpEx ($/yr)'),
        dcc.Input(id='maint_opex', type='number', value=0, style={'width': '100%'}),
        html.Label('Maintenance/OpEx Uncertainty (std dev, $)', style={'marginTop': '10px'}),
        dcc.Input(id='maint_opex_std', type='number', value=0, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'opex' in selected else 'none'})
    # Capex input
    capex_div = html.Div([
        html.Label('CapEx Investment ($)'),
        dcc.Input(id='capex', type='number', value=0, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'capex' in selected else 'none'})
    # Amortization input
    amort_div = html.Div([
        html.Label('Monthly Amortization ($)'),
        dcc.Input(id='amortization', type='number', value=0, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'amortization' in selected else 'none'})
    return [opex_div, capex_div, amort_div]

@app.callback(
    Output('buy_inputs', 'children'),
    [Input('buy_selector', 'value')]
 )
def display_buy_inputs(selected):
    # Always show both, but only use values if checked
    one_time_div = html.Div([
        html.Label('One-Time Purchase (Flat Fee) ($)'),
        dcc.Input(id='product_price', type='number', value=1000000, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'one_time' in selected else 'none'})
    subscription_div = html.Div([
        html.Label('Annual Subscription Cost ($/yr)'),
        dcc.Input(id='subscription_price', type='number', value=100000, style={'width': '100%'}),
        html.Label('Expected Annual Price Increase (%)', style={'marginTop': '10px'}),
        dcc.Input(id='subscription_increase', type='number', value=3, style={'width': '100%'})
    ], style={'marginBottom': '10px', 'display': 'block' if 'subscription' in selected else 'none'})
    return [one_time_div, subscription_div]

## 3. Core Calculation Logic
This section contains the main simulation and calculation functions used by the dashboard.

### Simulation Function

In [4]:
# --- Refactored Simulation Function: Discount All Future Cash Flows, Sophisticated Risk Modeling ---
def simulate_build_vs_buy(params):
    import numpy as np
    np.random.seed(42)  # Ensure reproducibility for consistent results
    # Core parameters (always required)
    build_timeline = float(params.get('build_timeline', 12.0) or 12.0)
    build_timeline_std = float(params.get('build_timeline_std', 0.0) or 0.0)
    fte_cost = float(params.get('fte_cost', 150000.0) or 150000.0)
    fte_cost_std = float(params.get('fte_cost_std', 0.0) or 0.0)
    fte_count = float(params.get('fte_count', 1.0) or 1.0)
    useful_life = float(params.get('useful_life', 5.0) or 5.0)
    prob_success = float(params.get('prob_success', 80.0) or 80.0) / 100.0
    wacc = float(params.get('wacc', 8.0) or 8.0) / 100.0

    # Non-core (optional) parameters: use only if included (nonzero)
    maint_opex = float(params.get('maint_opex', 0) or 0)
    maint_opex_std = float(params.get('maint_opex_std', 0) or 0)
    capex = float(params.get('capex', 0) or 0)
    amortization = float(params.get('amortization', 0) or 0)
    tech_risk = float(params.get('tech_risk', 0) or 0)
    vendor_risk = float(params.get('vendor_risk', 0) or 0)
    market_risk = float(params.get('market_risk', 0) or 0)
    misc_costs = float(params.get('misc_costs', 0) or 0)

    # Buy-side parameters (now both can be present)
    product_price = float(params.get('product_price', 0) or 0)
    subscription_price = float(params.get('subscription_price', 0) or 0)
    subscription_increase = float(params.get('subscription_increase', 0) or 0)
    buy_selector = params.get('buy_selector', [])

    n_sim = 1000

    # Ensure all std devs are non-negative
    build_timeline_std = max(build_timeline_std, 0)
    fte_cost_std = max(fte_cost_std, 0)
    maint_opex_std = max(maint_opex_std, 0)

    # Simulate build timeline and FTE cost with uncertainty
    if build_timeline_std > 0:
        timeline_samples = np.random.normal(build_timeline, build_timeline_std, n_sim)
    else:
        timeline_samples = np.full(n_sim, build_timeline)
    if fte_cost_std > 0:
        fte_cost_samples = np.random.normal(fte_cost, fte_cost_std, n_sim)
    else:
        fte_cost_samples = np.full(n_sim, fte_cost)
    # Remove any negative or NaN values from samples
    timeline_samples = np.where(np.isnan(timeline_samples) | (timeline_samples <= 0), build_timeline, timeline_samples)
    fte_cost_samples = np.where(np.isnan(fte_cost_samples) | (fte_cost_samples <= 0), fte_cost, fte_cost_samples)

    # --- Sophisticated Risk Modeling ---
    # Model each risk as a random variable (normal, mean=entered value, std=20% of mean)
    tech_risk_samples = np.random.normal(tech_risk, abs(tech_risk)*0.2, n_sim) if tech_risk > 0 else np.zeros(n_sim)
    vendor_risk_samples = np.random.normal(vendor_risk, abs(vendor_risk)*0.2, n_sim) if vendor_risk > 0 else np.zeros(n_sim)
    market_risk_samples = np.random.normal(market_risk, abs(market_risk)*0.2, n_sim) if market_risk > 0 else np.zeros(n_sim)
    # Clip to non-negative
    tech_risk_samples = np.clip(tech_risk_samples, 0, None)
    vendor_risk_samples = np.clip(vendor_risk_samples, 0, None)
    market_risk_samples = np.clip(market_risk_samples, 0, None)

    # --- Build Cost Calculation ---
    # 1. Labor cost (core):
    labor_cost = (timeline_samples / 12) * fte_cost_samples * fte_count
    # 2. CapEx: assumed upfront (no discounting)
    capex_cost = capex * np.ones(n_sim)
    # 3. Amortization: discounted monthly over build timeline
    amortization_pv = np.zeros(n_sim)
    if amortization > 0:
        for i in range(n_sim):
            months = int(np.round(timeline_samples[i]))
            amort_pv = 0
            for m in range(1, months+1):
                amort_pv += amortization / ((1 + wacc/12) ** m)
            amortization_pv[i] = amort_pv
    # 4. Maintenance/OpEx: discounted annually over useful life
    opex_pv = np.zeros(n_sim)
    if maint_opex > 0:
        if maint_opex_std > 0:
            opex_samples = np.random.normal(maint_opex, maint_opex_std, n_sim)
        else:
            opex_samples = np.full(n_sim, maint_opex)
        opex_samples = np.where(np.isnan(opex_samples) | (opex_samples < 0), maint_opex, opex_samples)
        for i in range(n_sim):
            opex_sum = 0
            for y in range(1, int(np.round(useful_life))+1):
                opex_sum += opex_samples[i] / ((1 + wacc) ** y)
            opex_pv[i] = opex_sum
    # 5. Misc Costs: simple addition (no simulation, no discounting)
    misc_costs_arr = misc_costs * np.ones(n_sim)
    # 6. Sum all build-side costs
    build_cost = labor_cost + capex_cost + amortization_pv + opex_pv + misc_costs_arr
    # 7. Apply risk factors (per simulation, multiplicative)
    total_risk = (1 + tech_risk_samples/100) * (1 + vendor_risk_samples/100) * (1 + market_risk_samples/100)
    build_cost = build_cost * total_risk
    # 8. Probability of success (expected cost adjustment)
    prob_success = np.clip(prob_success, 0.01, 1.0)
    build_cost = build_cost / prob_success
    # 9. Discount build cost to present value (WACC, over build timeline)
    discount_factor = (1 + wacc) ** (timeline_samples / 12)
    build_cost_pv = build_cost / discount_factor
    # Remove any NaN or inf values from build_cost_pv for robustness
    build_cost_pv = np.where(np.isnan(build_cost_pv) | np.isinf(build_cost_pv), np.mean(build_cost_pv[~np.isnan(build_cost_pv) & ~np.isinf(build_cost_pv)]), build_cost_pv)

    # --- Buy Cost Calculation (unchanged) ---
    buy_total_cost = 0.0
    if 'one_time' in buy_selector:
        buy_total_cost += product_price
    if 'subscription' in buy_selector:
        years = int(np.round(useful_life))
        cashflows = np.array([subscription_price * ((1 + subscription_increase) ** y) for y in range(years)])
        discount_factors = np.array([(1 + wacc) ** y for y in range(years)])
        buy_total_cost += np.sum(cashflows / discount_factors)

    # NPV calculation (difference, positive = build is better)
    npv = buy_total_cost - np.mean(build_cost_pv)
    # Recommendation
    recommendation = 'Build' if npv > 0 else 'Buy'
    # Output percentiles
    ci_low = np.percentile(build_cost_pv, 10)
    ci_high = np.percentile(build_cost_pv, 90)
    # Return all relevant results
    return {
        'expected_build_cost': float(np.mean(build_cost_pv)),
        'ci_low': float(ci_low),
        'ci_high': float(ci_high),
        'buy_total_cost': float(buy_total_cost),
        'npv': float(npv),
        'recommendation': recommendation,
        'cost_distribution': build_cost_pv.tolist(),
        'tech_risk': tech_risk,
        'vendor_risk': vendor_risk,
        'market_risk': market_risk
    }

### App Runner

In [5]:
# --- Dash Callback and App Runner ---
from dash import Input, Output, State
import plotly.graph_objs as go
import threading
import time

@app.callback(
    [Output("results", "children"), Output("cost_dist", "figure")],
    [Input("calc_btn", "n_clicks")],
    [State("build_timeline", "value"), State("build_timeline_std", "value"),
     State("fte_cost", "value"), State("fte_cost_std", "value"), State("fte_count", "value"),
     State("cap_percent", "value"),
     State("misc_costs", "value"),
     State("buy_selector", "value"),
     State("product_price", "value"),
     State("subscription_price", "value"), State("subscription_increase", "value"),
     State("useful_life", "value"), State("prob_success", "value"), State("wacc", "value"),
     State("risk_selector", "value"),
     State("tech_risk", "value"), State("vendor_risk", "value"), State("market_risk", "value"),
     State("cost_selector", "value"),
     State("maint_opex", "value"), State("maint_opex_std", "value"),
     State("capex", "value"), State("amortization", "value")]
 )
def update_results(n_clicks, build_timeline, build_timeline_std, fte_cost, fte_cost_std, fte_count, cap_percent, misc_costs,
                  buy_selector, product_price, subscription_price, subscription_increase, useful_life, prob_success, wacc,
                  risk_selector, tech_risk, vendor_risk, market_risk,
                  cost_selector, maint_opex, maint_opex_std, capex, amortization):
    if n_clicks == 0:
        return "Enter inputs and click Calculate.", go.Figure()

    def safe_float(val, default=0.0):
        try:
            return float(val) if val not in (None, "") else default
        except Exception:
            return default

    cost_selector = cost_selector or []
    risk_selector = risk_selector or []
    buy_selector = buy_selector or []

    # Defensive: ensure misc_costs is always a float (never None)
    misc_costs_val = safe_float(misc_costs, 0.0)

    # Always provide all core params, and set optional params to zero if unchecked or blank
    params = {
        'build_timeline': safe_float(build_timeline, 0.0),
        'build_timeline_std': safe_float(build_timeline_std, 0.0),
        'fte_cost': safe_float(fte_cost, 0.0),
        'fte_cost_std': safe_float(fte_cost_std, 0.0),
        'fte_count': safe_float(fte_count, 0.0),
        'product_price': safe_float(product_price, 0.0) if 'one_time' in buy_selector else 0.0,
        'subscription_price': safe_float(subscription_price, 0.0) if 'subscription' in buy_selector else 0.0,
        'subscription_increase': safe_float(subscription_increase, 0.0) / 100.0 if 'subscription' in buy_selector else 0.0,
        'useful_life': safe_float(useful_life, 0.0),
        'prob_success': safe_float(prob_success, 0.0),
        'wacc': safe_float(wacc, 0.0),
        'maint_opex': safe_float(maint_opex, 0.0) if 'opex' in cost_selector else 0.0,
        'maint_opex_std': safe_float(maint_opex_std, 0.0) if 'opex' in cost_selector else 0.0,
        'capex': safe_float(capex, 0.0) if 'capex' in cost_selector else 0.0,
        'amortization': safe_float(amortization, 0.0) if 'amortization' in cost_selector else 0.0,
        'tech_risk': safe_float(tech_risk, 0.0) if 'tech' in risk_selector else 0.0,
        'vendor_risk': safe_float(vendor_risk, 0.0) if 'vendor' in risk_selector else 0.0,
        'market_risk': safe_float(market_risk, 0.0) if 'market' in risk_selector else 0.0,
        'buy_selector': buy_selector,
        'misc_costs': misc_costs_val,
    }

    # Track which non-core parameters are used for display
    non_core_used = []
    if 'opex' in cost_selector: non_core_used.append('Annual Maintenance/OpEx')
    if 'capex' in cost_selector: non_core_used.append('CapEx Investment')
    if 'amortization' in cost_selector: non_core_used.append('Monthly Amortization')
    if 'tech' in risk_selector: non_core_used.append('Technical Risk')
    if 'vendor' in risk_selector: non_core_used.append('Vendor Risk')
    if 'market' in risk_selector: non_core_used.append('Market Risk')
    if 'one_time' in buy_selector: non_core_used.append('One-Time Purchase')
    if 'subscription' in buy_selector: non_core_used.append('Annual Subscription')

    try:
        results = simulate_build_vs_buy(params)
    except Exception as e:
        return html.Div([html.P(f"Error in calculation: {e}", style={"color": "red"})]), go.Figure()

    # Build results output
    rec = results['recommendation']
    rec_style = {
        'backgroundColor': '#fff700',
        'color': '#000',
        'padding': '2px 8px',
        'borderRadius': '4px',
        'fontWeight': 'bold',
        'fontSize': '1.1em',
        'marginLeft': '4px'
    }

    results_div = html.Div([
        html.P(f"Expected Build Cost: ${results['expected_build_cost']:,.0f}", style={"color": "#000"}),
        html.P(f"10th-90th Percentile Range: ${results['ci_low']:,.0f} - ${results['ci_high']:,.0f}", style={"color": "#000"}),
        html.P(f"Buy Total Cost: ${results['buy_total_cost']:,.0f}", style={"color": "#000"}),
        html.P(f"NPV (Build): ${results['npv']:,.0f}", style={"color": "#000"}),
        html.P([html.Span("Recommendation:", style={"color": "#000"}), html.Span(rec, style=rec_style)]),
        html.P(f"Assumptions: Uncertainty included for timeline, FTE cost, and maintenance/OpEx", style={"color": "#000"}),
        html.P(f"Risk Factors Applied: Technical {results['tech_risk']}%, Vendor {results['vendor_risk']}%, Market {results['market_risk']}%", style={"color": "#000"}),
        html.P(f"Optional Parameters Used: {', '.join(non_core_used) if non_core_used else 'None'}", style={"color": "#000", "fontStyle": "italic"})
    ])

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=results['cost_distribution'], nbinsx=50, name="Build Cost Distribution"))
    fig.add_vline(x=results['buy_total_cost'], line_dash="dash", line_color="red", annotation_text="Buy Cost", annotation_position="top right")
    fig.update_layout(title="Build Cost Distribution vs Buy Cost", xaxis_title="Total Cost ($)", yaxis_title="Frequency")

    return results_div, fig

### Feature Logic

In [6]:
# --- Callback to generate and serve CSV ---
import io
import csv
from dash import callback_context
@app.callback(
    Output("download_csv", "data"),
    [Input("download_btn", "n_clicks")],
    [State("build_timeline", "value"), State("build_timeline_std", "value"),
     State("fte_cost", "value"), State("fte_cost_std", "value"), State("fte_count", "value"),
     State("cap_percent", "value"),
     State("product_price", "value"),
     State("useful_life", "value"), State("prob_success", "value"), State("wacc", "value"),
     State("risk_selector", "value"),
     State("tech_risk", "value"), State("vendor_risk", "value"), State("market_risk", "value"),
     State("cost_selector", "value"),
     State("maint_opex", "value"), State("maint_opex_std", "value"),
     State("capex", "value"), State("amortization", "value"),
     State("results", "children")
    ]
)
def download_results(n_clicks, build_timeline, build_timeline_std, fte_cost, fte_cost_std, fte_count, cap_percent,
                    product_price, useful_life, prob_success, wacc,
                    risk_selector, tech_risk, vendor_risk, market_risk,
                    cost_selector, maint_opex, maint_opex_std, capex, amortization, results_div):
    if n_clicks == 0:
        return dash.no_update
    # Collect input parameters
    inputs = {
        "Build Timeline (months)": build_timeline,
        "Build Timeline Std Dev": build_timeline_std,
        "FTE Cost ($/yr)": fte_cost,
        "FTE Cost Std Dev": fte_cost_std,
        "FTE Count": fte_count,
        "Capitalization Percent (%)": cap_percent,
        "Product Price ($)": product_price,
        "Useful Life (years)": useful_life,
        "Probability of Success (%)": prob_success,
        "WACC (%)": wacc,
        "Risk Factors": risk_selector,
        "Technical Risk (%)": tech_risk,
        "Vendor Risk (%)": vendor_risk,
        "Market Risk (%)": market_risk,
        "Cost Parameters": cost_selector,
        "Annual Maintenance/OpEx ($/yr)": maint_opex,
        "Maintenance/OpEx Std Dev": maint_opex_std,
        "CapEx Investment ($)": capex,
        "Monthly Amortization ($)": amortization
    }
    # Parse results from the results_div (extract text from html.P children)
    results = []
    import dash.html as html
    if isinstance(results_div, html.Div):
        for child in results_div.children:
            if hasattr(child, 'children'):
                results.append(child.children)
            else:
                results.append(str(child))
    else:
        results.append(str(results_div))
    # Prepare CSV content
    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerow(["Input Parameter", "Value"])
    for k, v in inputs.items():
        writer.writerow([k, v])
    writer.writerow([])
    writer.writerow(["Result", "Value"])
    for r in results:
        writer.writerow([r, ""])
    csv_data = output.getvalue()
    return dict(content=csv_data, filename="build_buy_results.csv")

In [7]:
# --- Scenario Table Update Callback ---
from dash import dash_table
@app.callback(
    Output("scenario_table_container", "children"),
    [Input("scenario_store", "data")]
 )
def update_scenario_table(scenario_store):
    scenarios = scenario_store or []
    data = []
    for s in scenarios:
        row = {**s}
        data.append(row)
    display_columns = ["Scenario Name", "Expected Build Cost", "NPV", "Recommendation",
        "Build Timeline (mo)", "FTE Cost", "FTE Cost StdDev", "FTE Count", "Cap %", "Misc Costs", "Product Price", "Useful Life", "Prob Success", "WACC", "Tech Risk", "Vendor Risk", "Market Risk", "Maint/OpEx", "CapEx", "Amortization"]
    table = dash_table.DataTable(
        columns=[{"name": k, "id": k, "deletable": False} for k in display_columns],
        data=data,
        style_table={"overflowX": "auto", "minWidth": "100%", "maxWidth": "100%"},
        style_cell={
            "textAlign": "left",
            "fontFamily": "Segoe UI, Arial, sans-serif",
            "padding": "6px 8px",
            "whiteSpace": "normal",
            "maxWidth": "160px",
            "fontSize": "15px"
        },
        style_header={"backgroundColor": "#f4f6f8", "fontWeight": "bold"},
        style_data_conditional=[
            {
                "if": {"column_id": "Recommendation", "filter_query": "{Recommendation} = 'Build'"},
                "backgroundColor": "#d4efdf",
                "color": "#145a32",
                "fontWeight": "bold"
            },
            {
                "if": {"column_id": "Recommendation", "filter_query": "{Recommendation} = 'Buy'"},
                "backgroundColor": "#f9e79f",
                "color": "#7d6608",
                "fontWeight": "bold"
            }
        ],
        page_size=5,
        style_as_list_view=True,
        style_cell_conditional=[
            {"if": {"column_id": "Expected Build Cost"}, "minWidth": "120px", "width": "120px", "maxWidth": "140px"},
            {"if": {"column_id": "NPV"}, "minWidth": "80px", "width": "80px", "maxWidth": "100px"},
            {"if": {"column_id": "Recommendation"}, "minWidth": "110px", "width": "110px", "maxWidth": "120px"},
            {"if": {"column_id": "Scenario Name"}, "minWidth": "120px", "width": "120px", "maxWidth": "160px"},
            {"if": {"column_id": "Misc Costs"}, "minWidth": "100px", "width": "100px", "maxWidth": "120px"}
        ],
        row_deletable=False,
        editable=False,
        id="scenario_table"
    )
    return table

In [8]:
# --- Save Scenario Button Callback ---
@app.callback(
    Output("scenario_store", "data"),
    [Input("save_scenario_btn", "n_clicks")],
    [State("build_timeline", "value"), State("build_timeline_std", "value"),
     State("fte_cost", "value"), State("fte_cost_std", "value"), State("fte_count", "value"),
     State("cap_percent", "value"),
     State("misc_costs", "value"),
     State("buy_selector", "value"),
     State("product_price", "value"),
     State("subscription_price", "value"), State("subscription_increase", "value"),
     State("useful_life", "value"), State("prob_success", "value"), State("wacc", "value"),
     State("scenario_name", "value"),
     State("risk_selector", "value"),
     State("tech_risk", "value"), State("vendor_risk", "value"), State("market_risk", "value"),
     State("cost_selector", "value"),
     State("maint_opex", "value"), State("maint_opex_std", "value"),
     State("capex", "value"), State("amortization", "value"),
     State("scenario_store", "data"),
     State("results", "children")
    ]
)
def save_scenario(n_clicks, build_timeline, build_timeline_std, fte_cost, fte_cost_std, fte_count, cap_percent, misc_costs,
                  buy_selector, product_price, subscription_price, subscription_increase, useful_life, prob_success, wacc,
                  scenario_name,
                  risk_selector, tech_risk, vendor_risk, market_risk,
                  cost_selector, maint_opex, maint_opex_std, capex, amortization, scenario_store, results_div):
    if n_clicks == 0:
        return scenario_store or []

    def safe_float(val, default=0.0):
        try:
            return float(val) if val not in (None, "") else default
        except Exception:
            return default

    cost_selector = cost_selector or []
    risk_selector = risk_selector or []
    buy_selector = buy_selector or []

    # Always provide all core params, and set optional params to zero if unchecked or blank
    params = {
        'build_timeline': safe_float(build_timeline, 0.0),
        'build_timeline_std': safe_float(build_timeline_std, 0.0),
        'fte_cost': safe_float(fte_cost, 0.0),
        'fte_cost_std': safe_float(fte_cost_std, 0.0),
        'fte_count': safe_float(fte_count, 0.0),
        'product_price': safe_float(product_price, 0.0) if 'one_time' in buy_selector else 0.0,
        'subscription_price': safe_float(subscription_price, 0.0) if 'subscription' in buy_selector else 0.0,
        'subscription_increase': safe_float(subscription_increase, 0.0) / 100.0 if 'subscription' in buy_selector else 0.0,
        'useful_life': safe_float(useful_life, 0.0),
        'prob_success': safe_float(prob_success, 0.0),
        'wacc': safe_float(wacc, 0.0),
        'maint_opex': safe_float(maint_opex, 0.0) if 'opex' in cost_selector else 0.0,
        'maint_opex_std': safe_float(maint_opex_std, 0.0) if 'opex' in cost_selector else 0.0,
        'capex': safe_float(capex, 0.0) if 'capex' in cost_selector else 0.0,
        'amortization': safe_float(amortization, 0.0) if 'amortization' in cost_selector else 0.0,
        'tech_risk': safe_float(tech_risk, 0.0) if 'tech' in risk_selector else 0.0,
        'vendor_risk': safe_float(vendor_risk, 0.0) if 'vendor' in risk_selector else 0.0,
        'market_risk': safe_float(market_risk, 0.0) if 'market' in risk_selector else 0.0,
        'buy_selector': buy_selector,
        'misc_costs': safe_float(misc_costs, 0.0),
    }

    # Run simulation to get results for scenario (use same logic as calculate)
    try:
        results = simulate_build_vs_buy(params)
    except Exception:
        return scenario_store or []

    scenario_store = scenario_store or []
    scenario = {
        "Scenario Name": scenario_name or f"Scenario {len(scenario_store)+1}",
        "Expected Build Cost": f"{results['expected_build_cost']:,.0f}",
        "NPV": f"{results['npv']:,.0f}",
        "Recommendation": results['recommendation'],
        "Build Timeline (mo)": build_timeline,
        "FTE Cost": fte_cost,
        "FTE Cost StdDev": fte_cost_std,
        "FTE Count": fte_count,
        "Cap %": cap_percent,
        "Misc Costs": misc_costs,
        "Product Price": product_price,
        "Useful Life": useful_life,
        "Prob Success": prob_success,
        "WACC": wacc,
        "Tech Risk": tech_risk,
        "Vendor Risk": vendor_risk,
        "Market Risk": market_risk,
        "Maint/OpEx": maint_opex,
        "CapEx": capex,
        "Amortization": amortization
    }
    # Only add if not a duplicate of the last scenario
    if not scenario_store or scenario != scenario_store[-1]:
        scenario_store.append(scenario)

    return scenario_store

---

# 🚦 Launch the Dashboard

Below, the dashboard will launch. All logic and features are preloaded above for clarity and reproducibility.


# ℹ️ User Guidance: How to Use the Build vs. Buy Dashboard

This dashboard helps you make a rigorous, unbiased decision between building a technology solution in-house or buying it. Please review the following guidance to ensure your inputs are meaningful and your results are interpreted correctly.

## Key Parameters Explained

**Build Cost Parameters:**
- **Build Timeline (months):** How long you expect the build to take. Add uncertainty (std dev) if unsure.
- **FTE Cost ($/yr):** Fully loaded annual cost per engineer (salary + benefits). Add uncertainty (std dev) if unsure.
- **FTE Count:** Number of full-time engineers required.
- **Capitalization Percent (%):** Portion of labor cost that can be capitalized (for R&D tax purposes).
- **Annual Maintenance/OpEx ($/yr):** Ongoing annual costs to maintain the solution. Add uncertainty if needed.
- **CapEx Investment ($):** Upfront capital expenditures (hardware, licenses, etc.).
- **Monthly Amortization ($):** Any recurring monthly costs during the build phase.

**Buy Cost Parameters:**
- **One-Time Purchase (Flat Fee):** Upfront cost to buy the solution.
- **Annual Subscription ($/yr):** Ongoing subscription/license fees. You can specify an expected annual price increase (%).
- **Useful Life (years):** How long you expect the solution to be relevant.

**Risk Factors:**
- **Technical Risk (%):** Expected average cost overrun due to technical challenges. E.g., 0% = no risk, 10% = expect 10% average overrun, 100% = expect costs could double. The model simulates risk as a random variable (normal, mean = your input, std = 20% of mean, clipped to non-negative).
- **Vendor Risk (%):** Risk of delays or cost increases from third-party vendors. Interpreted the same way as technical risk.
- **Market Risk (%):** Risk of market changes making the solution less valuable or more costly. Interpreted the same way.

**Probability of Success (%):**
- The chance the build will succeed as planned. Lower values increase expected cost (model divides by this probability).

**WACC (%):**
- Weighted Average Cost of Capital. Used to discount all future cash flows to present value. Higher WACC means future costs are less valuable today, which can make buying (with more recurring costs) less attractive compared to building (with more upfront costs).

## How to Interpret and Select Risk Values
- **0%:** No risk; costs are expected to be exactly as estimated.
- **10%:** Mild risk; expect about 10% average cost overrun.
- **25%:** Moderate risk; expect about 25% average overrun.
- **50%:** High risk; costs could be 50% higher on average.
- **100%:** Extreme risk; costs could double on average.
- **Tip:** Use your organization's historical data or expert judgment. If unsure, start with 10–25% for technical risk, 0–10% for vendor/market risk unless you have reason to expect more.

## How the Model Works
- All future cash flows (amortization, OpEx, subscriptions) are discounted to present value using WACC.
- Risk factors are simulated as random variables for each run, so results reflect uncertainty.
- The dashboard runs 1,000 simulations and reports the expected cost, a percentile range, and a recommendation.
- The recommendation is based on which option (build or buy) has the lower expected present value cost.

## Best Practices
- Be realistic and transparent with your inputs.
- Review the percentile range to understand uncertainty, not just the average.

---

In [9]:
# --- App Runner ---
def run_dash():
    app.run(debug=False)

thread = threading.Thread(target=run_dash)
thread.start()
time.sleep(2)
print("Dashboard running at http://127.0.0.1:8050/")

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


Dashboard running at http://127.0.0.1:8050/
